## Assingment Submitted by:
### - Ankit Saini
### - 17BT30002


## Daily Rainfall Analysis

Two files are attached which contain daily rainfall data over India for 2010 and 2011. Both of them contain a 357x122 matrix (XR1 and XR) an a binary vector (ZR1 and ZR). The matrices contain rainfall amounts at 357 locations over India, on each day during the monsoon seasons of 2010 and 2011 (122 days from 1 June to 30 September). ZR1 and ZR are binary vectors which classify every day as 'rainy" (1) or non-rainy (0) based on the rainfall across the landmass.

1) Read the .mat files in Python and access the variables

2) Use a linear regression model to predict the rainfall XR(s,t) at any location 's' on day 't', using as predictor the rainfall at all other locations on the same day, and also rainfall at the same location on the previous 2 days [XR(1,t)....XR(s-1,t), XR(s+1,t),....XR(357,t), XR(s,t-1), XR(s,t-2)]. Use 2010 data for training.

Build such a model for s=42 (Mumbai), s=158 (Delhi), s= 299 (Kharagpur) [3 marks]

3) Use the same model to predict the rainfall at these 3 locations on each day of 2011.  Use values in XR as predictors. Compare the results with the true values and compute error for 3 locations separately. [1 marks]

4) Repeat the same process using LASSO linear regression. Using the coefficients, identify the top 5 predictors for each of the 3 locations. [2 marks]

5) Use Decision Tree on 2010 data to classify each day as 1 or 0 (as given in ZR1). For each day, use the 357-dimensional rainfall vector as feature vector. Report the 10 most discriminative features (i.e. locations) [3 marks]

6) Use this Decision Tree to classify each day of 2011 as 1 or 0. Report accuracy by comparing with ZR. [1 mark]

In [1]:
#Importing necessary Libraries
import scipy.io
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Lasso

#### 1) Read the .mat files in Python and access the variables

In [2]:
# Loading .mat files
mat_train = scipy.io.loadmat('2010rainfall.mat')
mat_tests = scipy.io.loadmat('2011rainfall.mat')

These are the keys present in given dataset "2010rainfall.mat"

In [3]:
print (mat_train.keys()) #required only 'XR1' & 'ZR1'
print (mat_tests.keys()) #required only 'XR' & 'ZR'

dict_keys(['__header__', '__version__', '__globals__', 'XR1', 'ZR1'])
dict_keys(['__header__', '__version__', '__globals__', 'XR', 'ZR'])


In [4]:
mat_train['XR1'].shape

(357, 122)

The shape of the matrix is 357 rows and 122 columns as shown above. Which contain rainfall amount for 357 different locations over india, on each day during the monsoon seasons of 2010 and 2011 (122 days from 1 June to 30 September)

In [5]:
# Creating dataframe using pandas
def dataframe(mat_file): 
    return pd.DataFrame(mat_file.transpose(), columns = list(range(1, 358))) #Creating dataframe


location = [42, 158, 299] #locations required for Mumbai, delhi, Kharagpur

In [25]:
# Dataframe for year 2010
rainfall10_df = dataframe(mat_train['XR1'])

#### 2) Use a linear regression model to predict the rainfall XR(s,t) at any location 's' on day 't', using as predictor the rainfall at all other locations on the same day, and also rainfall at the same location on the previous 2 days [XR(1,t)....XR(s-1,t), XR(s+1,t),....XR(357,t), XR(s,t-1), XR(s,t-2)]. Use 2010 data for training.

#### Linear Regression Model

In [7]:
# Training the model
def linear_regression(X, Y): #By inbuilt Linear Regression Model
    reg = LinearRegression()
    reg.fit(X, Y)
    var = reg.coef_[0] 
    M = reg.intercept_[0]
    return var, M

In [8]:
constant = []
coff_weights = []
for j in location:
    coff, cons = linear_regression(rainfall10_df.drop(j, 1), pd.DataFrame(rainfall10_df[j], columns = [j]))
    coff_weights.append(coff)
    constant.append(cons)

Weights of every location show the importance of there rainfall data for the prediction of rainfall at any other location 's' on day 't'. Prediction can be determine by using the below define prediction function. 

#### 3) Use the same model to predict the rainfall at these 3 locations on each day of 2011. Use values in XR as predictors. Compare the results with the true values and compute error for 3 locations separately. 

In [9]:
# Creating dataframe for year 2011 using above defined function
rainfall11_df = dataframe(mat_tests['XR'])

Prediction using linear regression model

In [10]:
# Prediction on the test dataset, Rainfall analysis
class prediction():
    
    def predicting_rainfall(self, X, Y, Z, W):
        prediction = []
        for row in range(122):
            if row < 2:
                prediction.append(np.dot(np.array(X.loc[row]), Y) + Z)
            else:
                #rainfall11_df[W][row-1], rainfall at day - 1
                #rainfall11_df[W][row-2], rainfall at day - 2
                prediction.append(np.dot(np.array(X.loc[row]), Y) + Z + rainfall11_df[W][row-1] + rainfall11_df[W][row-2])
            
        return prediction

In [11]:
delhi = prediction()
mumbai = prediction()
kharagpur = prediction()

mum_pred = mumbai.predicting_rainfall(rainfall11_df.drop(42, 1), coff_weights[0], constant[0], location[0])
del_pred = delhi.predicting_rainfall(rainfall11_df.drop(158, 1), coff_weights[1], constant[1], location[1])
kgp_pred = kharagpur.predicting_rainfall(rainfall11_df.drop(299, 1), coff_weights[2], constant[2], location[2])

In [12]:
#Calculating the error
print ('Error for data on Mumbai is', sqrt(mean_squared_error(mum_pred, list(rainfall11_df[42]))))
print ('Error for data on Delhi is', sqrt(mean_squared_error(del_pred, list(rainfall11_df[158]))))
print ('Error for data on kharagpur is', sqrt(mean_squared_error(kgp_pred, list(rainfall11_df[299]))))

Error for data on Mumbai is 88.63079322866871
Error for data on Delhi is 35.07074163173628
Error for data on kharagpur is 48.19770964494719


#### 4) Repeat the same process using LASSO linear regression. Using the coefficients, identify the top 5 predictors for each of the 3 locations.

In [13]:
#Reapeating using LASSO
mumbai = Lasso(alpha = 0.1)
mumbai.fit(rainfall10_df.drop(42, 1),pd.DataFrame(rainfall10_df[42], columns = [42]))

delhi = Lasso(alpha = 0.1)
delhi.fit(rainfall10_df.drop(158, 1),pd.DataFrame(rainfall10_df[158], columns = [158]))

kharagpur = Lasso(alpha = 0.1)
kharagpur.fit(rainfall10_df.drop(299, 1),pd.DataFrame(rainfall10_df[299], columns = [299]))

/home/moraxella/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 79.19353607982558, tolerance: 14.362556867025141
  model = cd_fast.enet_coordinate_descent(
/home/moraxella/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 70.53983749824799, tolerance: 2.8048406879295706
  model = cd_fast.enet_coordinate_descent(
/home/moraxella/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 36.08922948156892, tolerance: 2.2319624113395755
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [14]:
# Modifying the indices accordingly to the missing columns 
def top_five(lst, x):
    for i in range(len(lst)):
        if lst[i] > x:
            lst[i] = lst[i] + 1
            
    return lst

In [16]:
#Getting top 5 predictors

coeff_m = mumbai.coef_
index_m = sorted(range(len(coeff_m)), key = lambda i: coeff_m[i], reverse = True)[:5]

coeff_d = delhi.coef_
index_d = sorted(range(len(coeff_d)), key = lambda i: coeff_d[i], reverse = True)[:5]

coeff_k = kharagpur.coef_
index_k = sorted(range(len(coeff_k)), key = lambda i: coeff_k[i], reverse = True)[:5]

In [17]:
print ('Five Most Important Predictor for Mumbai are on indices', top_five(index_m, 42))
print ('Five Most Important Predictor for Delhi are on indices', top_five(index_d, 158))
print ('Five Most Important Predictor for Kharagpur are on indices', top_five(index_k, 299))

Five Most Important Predictor for Mumbai are on indices [25, 63, 40, 130, 200]
Five Most Important Predictor for Delhi are on indices [252, 127, 183, 304, 158]
Five Most Important Predictor for Kharagpur are on indices [305, 125, 278, 298, 96]


#### 5) Use Decision Tree on 2010 data to classify each day as 1 or 0 (as given in ZR1). For each day, use the 357-dimensional rainfall vector as feature vector. Report the 10 most discriminative features (i.e. locations) 

In [18]:
# Getting the data set
y_train = mat_train['ZR1'].transpose()

In [19]:
# Training the Model
regressor = DecisionTreeClassifier()
regressor.fit(rainfall10_df, y_train)

DecisionTreeClassifier()

In [20]:
# Getting feature importances
imp_feat = regressor.feature_importances_

# Picking the values for feature importance
index = sorted(range(len(imp_feat)), key=lambda i: imp_feat[i], reverse=True)[:10]
print(f"Ten most important indices are {index}")

Ten most important indices are [184, 252, 163, 26, 204, 159, 65, 172, 69, 0]


#### 6) Use this Decision Tree to classify each day of 2011 as 1 or 0. Report accuracy by comparing with ZR.

In [21]:
# Getting the data set
y_test = mat_tests['ZR'].transpose()

# Predicting 
y_pred = regressor.predict(rainfall11_df)

In [22]:
y_pred

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [23]:
#Calculating the accuracy of deision tree
acc = accuracy_score(y_test, y_pred)
print ("Accuracy by comparing the ZR with predicted values of test cases", acc)

Accuracy by comparing the ZR with predicted values of test cases 0.7049180327868853
